<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/RNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

In [2]:
treated_data_PB = pd.read_csv('https://raw.githubusercontent.com/mariabandeira/AdministracaoPublica/refs/heads/main/CadUnico/BasesPB/cadunico_pb_cleaned.csv', sep=',', encoding='utf-8')

In [3]:
treated_data_PB.dropna(inplace=True)
treated_data_PB.drop_duplicates(inplace=True)

### Treinando o modelo RNN

In [4]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_PB['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    60.96
0    39.04
Name: proportion, dtype: float64


In [5]:
# separar X (features) e y (target)
X = treated_data_PB.drop('marc_pbf', axis=1)
y = treated_data_PB['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    49935
1    49935
Name: count, dtype: int64


In [6]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[-1.94675229  0.25998787 -0.66351538  0.         -1.13901679  0.21086124
   1.58901493 -0.30371528 -0.63344419 -0.68283037 -0.32494349 -0.64034823
  -0.30392907 -1.16344252  0.          0.08346976  0.07556882]
 [-0.56513795 -0.13902563 -0.66351538  0.         -1.13901679 -1.08954091
  -0.53557624 -0.30371528 -0.63344419 -0.68283037 -0.32494349 -0.64034823
  -0.30392907 -1.16344252  0.          0.08346976  1.48597033]
 [-0.56513795  0.18154932 -0.66351538  0.          0.17970507  0.21086124
   1.58901493 -0.30371528 -0.63344419 -0.68283037 -0.32494349 -0.64034823
  -0.30392907 -1.16344252  0.          0.08346976  0.78076958]
 [ 0.81647638  0.31796419 -0.66351538  0.          0.839066    0.21086124
  -0.53557624 -0.30371528 -0.63344419 -0.68283037 -0.32494349 -0.64034823
  -0.30392907 -1.16344252  0.          0.08346976 -0.62963193]
 [-0.56513795  0.42709609 -0.66351538  0.          1.49842693  1.51126339
   1.58901493 -0.30371528 -0.63344419 -0

In [7]:
# Ajustar formato para a RNN (samples, time steps, features)
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [9]:
# Construir a RNN
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7951 - loss: 0.4410 - val_accuracy: 0.8594 - val_loss: 0.3640
Epoch 2/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8425 - loss: 0.3723 - val_accuracy: 0.8569 - val_loss: 0.3683
Epoch 3/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8450 - loss: 0.3668 - val_accuracy: 0.8580 - val_loss: 0.3718
Epoch 4/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8430 - loss: 0.3687 - val_accuracy: 0.8643 - val_loss: 0.3575
Epoch 5/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8447 - loss: 0.3662 - val_accuracy: 0.8623 - val_loss: 0.3584
Epoch 6/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step - accuracy: 0.8444 - loss: 0.3648 - val_accuracy: 0.8637 - val_loss: 0.3602
Epoch 7/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8460 - loss: 0.3631 - val_accuracy: 0.8602 - val_loss: 0.3646
Epoch 8/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8469 - loss: 0

In [12]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

1708/1708 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.76      0.81     21189
           1       0.86      0.93      0.89     33467

    accuracy                           0.86     54656
   macro avg       0.87      0.85      0.85     54656
weighted avg       0.87      0.86      0.86     54656



### Selecionando features com SelectKBests

In [14]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'qtd_comodos_domic_fam',
       'cod_agua_canalizada_fam', 'cod_abaste_agua_domic_fam', 'qtde_pessoas'],
      dtype='object')


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 3 14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [16]:
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [17]:
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train_scaled.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.8186 - loss: 0.4155 - val_accuracy: 0.8594 - val_loss: 0.3708
Epoch 2/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.8435 - loss: 0.3713 - val_accuracy: 0.8636 - val_loss: 0.3659
Epoch 3/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8461 - loss: 0.3693 - val_accuracy: 0.8637 - val_loss: 0.3637
Epoch 4/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8457 - loss: 0.3684 - val_accuracy: 0.8627 - val_loss: 0.3650
Epoch 5/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8457 - loss: 0.3656 - val_accuracy: 0.8657 - val_loss: 0.3592
Epoch 6/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.8455 - loss: 0.3682 - val_accuracy: 0.8624 - val_loss: 0.3652
Epoch 7/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8459 - loss: 0.3673 - val_accuracy: 0.8641 - val_loss: 0.3663
Epoch 8/20
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8464 - loss: 0

In [20]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

1708/1708 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.76      0.81     21189
           1       0.86      0.93      0.89     33467

    accuracy                           0.86     54656
   macro avg       0.86      0.84      0.85     54656
weighted avg       0.86      0.86      0.86     54656

